In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Load the data
survey_data  = pd.read_csv("/Users/vaibhavrangan/Downloads/Stat_303-1/Data/survey_data_clean.csv")
survey_data.shape

(192, 51)

## Q1: In the previous two iterations, students used the following two ways to create dummy variables from categorical variables.

#### Method 1: 

In [3]:
# # get dummy variables for all collumns except  the Timestamp column
survey_dummies_1 = pd.get_dummies(survey_data, columns = survey_data.columns[1:], dtype=int)

# # print out the number of columns in the new dataframe
print(survey_dummies_1.shape[1])
print(survey_dummies_1.head())


915
                      Timestamp  fav_alcohol_Beer  \
0   2022/09/13 1:43:34 pm GMT-5                 0   
1   2022/09/13 5:28:17 pm GMT-5                 0   
2   2022/09/13 7:56:38 pm GMT-5                 0   
3  2022/09/13 10:34:37 pm GMT-5                 0   
4   2022/09/14 4:46:19 pm GMT-5                 0   

   fav_alcohol_Hard liquor/Mixed drink  fav_alcohol_I don't drink  \
0                                    0                          1   
1                                    1                          0   
2                                    1                          0   
3                                    1                          0   
4                                    0                          1   

   fav_alcohol_Wine  parties_per_month_0.0  parties_per_month_1.0  \
0                 0                      0                      1   
1                 0                      0                      0   
2                 0                      0             

#### Method 2: 

In [4]:
survey_dummies2=pd.get_dummies(survey_data.iloc[:,1:], dtype='int')

# # print out the number of columns in the new dataframe
print(survey_dummies2.shape[1])
print(survey_dummies2.columns)

308
Index(['parties_per_month', 'love_first_sight', 'num_insta_followers',
       'expected_marriage_age', 'expected_starting_salary',
       'minutes_ex_per_week', 'sleep_hours_per_day',
       'farthest_distance_travelled', 'fav_number', 'internet_hours_per_day',
       ...
       'region_of_residence_Midwest', 'region_of_residence_Northeast',
       'region_of_residence_Southeast', 'region_of_residence_Southwest',
       'region_of_residence_West', 'political_affliation_Democrat',
       'political_affliation_Independent',
       'political_affliation_No affiliation', 'political_affliation_Other',
       'political_affliation_Republican'],
      dtype='object', length=308)


## Q1: which approch is correct? Use your code to show why the other one is incorrect?

In [5]:
print("Columns in first approach but not in second:", set(survey_dummies_1.columns) - set(survey_dummies2.columns))
print("Columns in second approach but not in first:", set(survey_dummies2.columns) - set(survey_dummies_1.columns))


Columns in first approach but not in second: {'internet_hours_per_day_9.0', 'high_school_GPA_4.01', 'NU_GPA_3.76', 'parties_per_month_5.0', 'height_76.0', 'num_insta_followers_300.0', 'internet_hours_per_day_100.0', 'internet_hours_per_day_210.0', 'farthest_distance_travelled_4284.0', 'height_father_67.7', 'fav_number_24.0', 'minutes_ex_per_week_240.0', 'num_insta_followers_612.0', 'farthest_distance_travelled_6887.0', 'farthest_distance_travelled_5000.0', 'parties_per_month_0.0', 'fav_number_3.0', 'high_school_GPA_4.97', 'num_insta_followers_2400.0', 'farthest_distance_travelled_6583.0', 'height_56.0', 'farthest_distance_travelled_7740.0', 'num_clubs_0', 'expected_marriage_age_30.0', 'farthest_distance_travelled_8000.0', 'farthest_distance_travelled_3000.0', 'num_insta_followers_60.0', 'fav_number_25.0', 'math_is_genetic_0', 'love_first_sight_1', 'height_father_68.0', 'only_child_0', 'farthest_distance_travelled_7353.0', 'high_school_GPA_4.58', 'NU_GPA_3.8', 'minutes_ex_per_week_375.0

The second method is correct because it only creates dummy variables for categorical variables whereas the first method creates different columns for every unique value of numerical variables, creating many more columns than the second method.

In [6]:
spotify_data = pd.read_csv("/Users/vaibhavrangan/Downloads/Stat_303-1/Data/spotify_data.csv")
spotify_data.columns

Index(['artist_followers', 'genres', 'artist_name', 'artist_popularity',
       'track_name', 'track_popularity', 'duration_ms', 'explicit',
       'release_year', 'danceability', 'energy', 'key', 'loudness', 'mode',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo', 'time_signature'],
      dtype='object')

In [7]:
spotify_data['energy_lvl'] = pd.qcut(spotify_data['energy'], q=2, labels=['Low energy', 'High energy'])
genre_energy_count = spotify_data.groupby(['genres', 'energy_lvl']).size().reset_index(name='num_observations')
genre_energy_count['percentage'] = (genre_energy_count['num_observations'] / genre_energy_count['num_observations'].sum()) * 100
highest_percentage_combo = genre_energy_count.loc[genre_energy_count['percentage'].idxmax()]
print(highest_percentage_combo)

genres                      pop
energy_lvl          High energy
num_observations          36699
percentage              15.0907
Name: 19, dtype: object


In [9]:
rock = spotify_data[(spotify_data['genres'] == 'rock')]
rock_artists_grouped = rock.groupby('artist_name')['artist_popularity'].mean()
# sort rock_artists_grouped by artist_popularity
rock_artists_grouped.sort_values(ascending=False).head(5)

artist_name
Queen                89.0
Imagine Dragons      89.0
The Beatles          88.0
Arctic Monkeys       87.0
Twenty One Pilots    86.0
Name: artist_popularity, dtype: float64

In [ ]:
# Find the top 5 artists by 'artist_popularity' in each genre
top_artists_per_genre = spotify_data.sort_values('artist_popularity', ascending=False).groupby('genres')['artist_name'].unique().apply(lambda x: x[:5])

# Filter the dataset to only include tracks by these top artists
top_tracks = spotify_data[spotify_data.apply(lambda row: row['artist_name'] in top_artists_per_genre.get(row['genres'], []), axis=1)]

# Calculate the number of unique tracks contributed by these top artists
unique_tracks_top_artists = top_tracks['track_name'].nunique()

# Calculate the total number of unique tracks in the dataset
total_unique_tracks = spotify_data['track_name'].nunique()

# Calculate the percentage
print((unique_tracks_top_artists / total_unique_tracks) * 100)



5.784114275117673
